In [ ]:
from MetaWorld.utilsMW.trainer import ActiveCritic, ActiveCriticArgs
from stable_baselines3.common.torch_layers import CombinedExtractor
from MetaWorld.utilsMW.model_setup_obj import NetworkSetup
from stable_baselines3 import PPO, SAC
from stable_baselines3.common.vec_env import DummyVecEnv
import gym
from MetaWorld.searchTest.utils import MyEnv, ToyExpertModel, sample_expert_transitions, benchmark_policy, LearnWrapper, train_policy, LearnWrapper, parse_sampled_transitions, VecExtractor
import torch
import zipfile
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
import importlib
import numpy as np
from RlBaselines3Zoo import enjoy
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.ppo import MlpPolicy

from imitation.algorithms import bc
from imitation.data import rollout
from imitation.data.wrappers import RolloutInfoWrapper

from stable_baselines3.common.policies import MultiInputActorCriticPolicy, ActorCriticPolicy, BaseModel
from stable_baselines3.common.torch_layers import (
    BaseFeaturesExtractor,
    CombinedExtractor,
    FlattenExtractor,
    NatureCNN,
    create_mlp,
    get_actor_critic_arch,
)
from sb3_contrib.tqc.tqc import TQC
from sb3_contrib.tqc.policies import MultiInputPolicy
import numpy as np
import torch as th
import os

from MetaWorld.searchTest.utils import train_policy

import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

model = enjoy.main(inpt_args="--algo tqc --env FetchPickAndPlace-v1 --folder /home/hendrik/Documents/master_project/Code/RlBaselines3Zoo/rl-trained-agents -n 300 --ret_model True")
def sample_expert_transitions():
    expert = model

    print("Sampling expert transitions.")
    rollouts = rollout.rollout(
        expert,
        DummyVecEnv([lambda: RolloutInfoWrapper(model.env.envs[0])]),
        rollout.make_sample_until(min_timesteps=None, min_episodes=100),
        unwrap=True,
        exclude_infos=False,
    )
    return rollout.flatten_trajectories(rollouts)

tp = '/home/hendrik/Documents/master_project/LokalData/ImitationLearning/transitions_rew_100'

#transitions = sample_expert_transitions()
#torch.save(transitions, tp)
transitions = torch.load('/home/hendrik/Documents/master_project/LokalData/ImitationLearning/transitions_rew_100')



In [ ]:
from MetaWorld.metaworld.policies.sawyer_pick_place_v2_policy import SawyerPickPlaceV2Policy
from metaworld.envs import ALL_V2_ENVIRONMENTS_GOAL_OBSERVABLE
from MetaWorld.utilsMW.makeTrainingData import make_policy_dict
from stable_baselines3.common.policies import ActorCriticPolicy
from stable_baselines3.common.on_policy_algorithm import OnPolicyAlgorithm
from typing import Union, Type, Optional
from gym.wrappers import TimeLimit

from stable_baselines3.common.type_aliases import GymEnv
policy_dict = make_policy_dict()
env_tag = 'reach'
max_episode_steps = 5
gt_policy = policy_dict[env_tag]
pape = ALL_V2_ENVIRONMENTS_GOAL_OBSERVABLE[policy_dict[env_tag][1]]()
pape._freeze_rand_vec = False
timelimit = TimeLimit(env=pape, max_episode_steps=max_episode_steps)
dv1 = DummyVecEnv([lambda: RolloutInfoWrapper(timelimit)])
class ImitationLearningWrapper:
    def __init__(self, policy, env:GymEnv):
        self.observation_space = env.observation_space
        self.action_space = env.action_space
        self.policy = policy

    def predict(self, obsv, deterministic=None):
        actions = []
        for obs in obsv:
            actions.append(self.policy.get_action(obs))
        return actions
IGTP = ImitationLearningWrapper(policy=gt_policy[0], env=dv1).predict

def new_epoch(current_obs, check_obsvs):
    result =  not th.equal(current_obs.reshape(-1)[-3:], check_obsvs.reshape(-1)[-3:])
    return result
    
class DummyExtractor:
    def __init__(self):
        pass
    def forward(self, features):
        if type(features) is np.ndarray:
            features = th.tensor(features)
        return features

def sample_expert_transitions(expert):
    print("Sampling expert transitions.")
    rollouts = rollout.rollout(
        expert,
        dv1,
        rollout.make_sample_until(min_timesteps=None, min_episodes=5),
        unwrap=True,
        exclude_infos=False,
    )
    return rollout.flatten_trajectories(rollouts)
transitions = sample_expert_transitions(IGTP)

In [ ]:
from searchTest.utils import make_counter_embedding
from searchTest.utils import get_num_bits


make_counter_embedding(torch.tensor([4, 1]), 3)
get_num_bits

In [ ]:
def strip_transitions(transitions):
    observations = np.array(transitions.obs)

def strip_obs(obs):
    proto_obs = obs[0]
    num_bits = get_num_bits(len(obs))
    counter_embedding = make_counter_embedding(torch.arange(len(obs)), num_bits)
    counter_embedding
    

In [ ]:
def parse_sampled_transitions(transitions, new_epoch, extractor):
    observations = []
    actions = []
    infos =[]
    next_observations = []
    dones = []
    rewards = []
    epch_actions = []
    epch_observations = []
    check_obsvs = extractor.forward(transitions[0]['obs'])
    for i in range(len(transitions)):
        current_obs = extractor.forward(features=transitions[i]['obs'])
        next_observation = extractor.forward(features=transitions[i]['next_obs'])

        if new_epoch(current_obs, check_obsvs):
            rewards.append(epch_success)
            check_obsvs = current_obs
            observations.append(torch.cat([*epch_observations], dim=0).unsqueeze(0))
            actions.append(epch_actions)
            next_observations.append(torch.cat([*epch_next_observations], dim=0).unsqueeze(0))
            dones.append(epch_dones)
            infos.append(epch_infos)

            epch_actions = []
            epch_observations = []
            epch_next_observations = []
            epch_dones = []
            epch_infos = []
        
        infos = transitions[i]['infos']
        if 'is_success' in infos:
            success = transitions[i]['infos']['is_success']
        elif 'success' in infos:
            success = transitions[i]['infos']['success']

        epch_actions.append(transitions[i]['acts'])
        epch_observations.append(current_obs.unsqueeze(0))
        
    observations.append(torch.cat([*epch_observations], dim=0).unsqueeze(0))
    actions.append(epch_actions)
    rewards.append(success)


    actions = torch.tensor(np.array(actions), dtype=torch.float)
    observations = torch.cat([*observations], dim=0).type(torch.float)
    rewards = torch.tensor(np.array(rewards), dtype=torch.float)
    return actions, observations, rewards

In [ ]:
pape = ALL_V2_ENVIRONMENTS_GOAL_OBSERVABLE[policy_dict[env_tag][1]]()
expert = policy_dict[env_tag][0]
success = False
obsv = pape.reset()
i = 0
while not success:
    act = expert.get_action(obsv)
    obsv, _, _, info = pape.step(act)
    success = info['success']
    i+= 1

In [ ]:
args_obj = ActiveCriticArgs()
args_obj.set_batchsize(32)
args_obj.set_data_path(path='/home/hendrik/Documents/master_project/LokalData/')
args_obj.set_device(device='cuda')
args_obj.set_feature_extractor(DummyExtractor())
args_obj.set_log_name('reach mitation')
args_obj.set_meta_optimizer_lr(1e-1)
args_obj.set_mlr(5e-5)
args_obj.set_lr(5e-5)
args_obj.set_network_setup(NetworkSetup())
args_obj.set_tboard(True)
args_obj.set_demonstrations(demonstrations=transitions)
args_obj.set_n_steps(n_steps=10000)
args_obj.set_epoch_len(epoch_len=max_episode_steps)
args_obj.set_imitation_phase(True)
args_obj.set_weight_decay(1e-2)
args_obj.set_new_epoch(new_epoch=new_epoch)
args_obj.set_eval_epochs(epochs=5)
args_obj.set_opt_steps(2)
args_obj.set_complete_modulo(1)
args_obj.set_observable(True)

ac = ActiveCritic(
    policy=None,
    env=dv1,
    args_obj=args_obj,
    learning_rate=5e-5,
    extractor=DummyExtractor()
)


In [15]:
kwargs = dict(weight_decay=1e-1)
a = torch.optim.AdamW([torch.ones([3], requires_grad=True)], lr=1, **kwargs)

In [ ]:
#ac.network.loadNetworkFromFile(path='/home/hendrik/Documents/master_project/LokalData/Data/Model/GT pickplace V2 25/last/')

In [ ]:
train_policy(trainer=ac, 
learn_fct=ac.learn, 
val_env=ac.env, 
logname=args_obj.logname, 
path='/home/hendrik/Documents/master_project/LokalData/TransformerImitationLearning/',
n_epochs=1000,
n_steps=1,
eval_epochs=1,
step_fct=lambda i:i+1,
new_epoch=new_epoch,
extractor=DummyExtractor()
)

In [ ]:
a, b, c = ac.network.sample_new_episode(policy=ac.policy, env=dv1, episodes=5, add_data=False)

In [ ]:
r, t = benchmark_policy(
    policy=ac.policy, 
    path='/home/hendrik/Documents/master_project/LokalData/Data/Model/GT pickplace V2 25/',
    logname='reload test',
    eval_epochs=20,
    val_env=dv1,
    stepid=1,
    best_reward= -10,
    new_epoch=new_epoch,
    extractor=DummyExtractor(),
    save_model=False
    )

In [ ]:
trans = sample_expert_transitions(ac.policy)

In [ ]:
a,b,c = parse_sampled_transitions(transitions=trans, new_epoch=new_epoch, extractor=DummyExtractor())

In [ ]:
print(b.shape)

In [ ]:
print(c.shape)

In [ ]:
policy_kwargs = dict(net_arch=dict(pi=[64, 64], qf=[400, 300]))
# Create the agent
model = SAC("MlpPolicy", timelimit, verbose=1, policy_kwargs=policy_kwargs)
model.device

In [ ]:
bc_policy = ActorCriticPolicy(observation_space=timelimit.observation_space, action_space=timelimit.action_space, lr_schedule=lambda a:1, net_arch=[512,512,512])

In [ ]:
env = dv1
bc_trainer = bc.BC(
    observation_space=env.observation_space,
    action_space=env.action_space,
    demonstrations=transitions,
    policy=bc_policy,
    device='cuda'
)

In [ ]:
train_policy(
    trainer=bc_trainer,
    learn_fct=bc_trainer.train,
    val_env=dv1,
    logname='GT 10 V2 ActorCritic',
    path='/home/hendrik/Documents/master_project/LokalData/ImitationLearning/',
    n_epochs=1000,
    n_steps=50,
    eval_epochs=100,
    step_fct=lambda i: i+1,
    new_epoch=new_epoch,
    extractor=DummyExtractor()
)

In [ ]:
benchmark_policy(
    bc_policy, 
    path='/home/hendrik/Documents/master_project/LokalData/ImitationLearning/',
    logname='BC test gt pick place V2',
    eval_epochs=300,
    val_env=dv1,
    stepid=1,
    best_reward=-1,
    new_epoch=new_epoch,
    extractor=DummyExtractor(),
    save_model=False,
    )

In [ ]:
#ac.policy.load_state_dict(torch.load('/home/hendrik/Documents/master_project/LokalData/TransformerImitationLearning/First integrated test/best_modeltensor(-0.9200)'))

In [ ]:
train_policy(trainer=ac, 
learn_fct=ac.learn, 
val_env=ac.env, 
logname='GT pick place V2', 
path='/home/hendrik/Documents/master_project/LokalData/TransformerImitationLearning/',
n_epochs=1000,
n_steps=1,
eval_epochs=1,
step_fct=lambda i:i+1,
new_epoch=new_epoch,
extractor=DummyExtractor()
)

In [ ]:
ac.learn(n_epochs=1)

In [ ]:
ac.network.runValidation(quick=False, pnt=True, epoch=1, save=False, complete=True)